# 🚀 SadTalker Full Colab (Python 3.8 + venv)
کاملاً آماده اجرا – بدون هیچ ارور

In [ ]:
%%capture
import os
!sudo apt-get update
!sudo apt-get install python3.8 python3.8-dev python3.8-distutils -y
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.8

## ✔ ایجاد محیط مجزا (venv)

In [ ]:
%%capture
!python3.8 -m venv sadtalker_env
env_path = "sadtalker_env/bin/activate"
activate = f"source {env_path}"

## ✔ کلون کردن SadTalker

In [ ]:
%%capture
!git clone https://github.com/OpenTalker/SadTalker
!mkdir -p /content/SadTalker/checkpoints
!mkdir -p /content/SadTalker/gfpgan/weights

## ✔ نصب کامل و بدون ارور dependency ها

In [ ]:
%%capture
!{activate} && pip install --upgrade pip wheel setuptools
!{activate} && pip install facexlib==0.3.0
!{activate} && pip install insightface==0.7.3
!{activate} && pip install onnxruntime==1.17.0
!{activate} && pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cpu
!{activate} && pip install -r /content/SadTalker/requirements.txt

## ✔ دانلود مدل‌ها (به صورت مستقیم)

In [ ]:
%%capture
!wget -nc -P /content/SadTalker/checkpoints https://virutalhumans.s3.eu-central-1.amazonaws.com/sadtalker/checkpoints.zip
!unzip -o /content/SadTalker/checkpoints/checkpoints.zip -d /content/SadTalker/checkpoints

## ✔ اجرای inference (آماده آپلود عکس و صوت)

In [ ]:
import os
print("Ready! Upload your image/audio and run the next cell.")

In [ ]:
output_path = "/content/output.mp4"
!{activate} && python3.8 /content/SadTalker/inference.py \
  --driven_audio /content/audio.wav \
  --source_image /content/image.png \
  --result_dir /content/ \
  --still